In [1]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torchvision.transforms as tt
from torch.utils.data import DataLoader, ConcatDataset
import torchvision

stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
batch_size = 1024


def get_transformations():
    return [
        tt.Compose([tt.ToTensor(), tt.Normalize(*stats)]),
        tt.Compose([tt.RandomCrop(32, padding=4, padding_mode='reflect'), 
                        tt.RandomHorizontalFlip(), 
                        tt.ToTensor(), 
                        tt.Normalize(*stats,inplace=True)]),
        tt.Compose([tt.RandomCrop(32, padding=4, padding_mode='reflect'), 
                         tt.RandomHorizontalFlip(p=0.5), 
                         tt.RandomVerticalFlip(p=0.5),
                         tt.ToTensor(), 
                         tt.Normalize(*stats,inplace=True)]),
        tt.Compose([tt.RandomCrop(32, padding=4, padding_mode='reflect'), 
                         tt.ColorJitter(brightness=0.4, 
                             contrast=0.5,
                             saturation=0.4, 
                             hue=0.3),
                         tt.GaussianBlur(kernel_size=(11, 21), sigma=(5, 50)),
                         tt.ToTensor(), 
                         tt.Normalize(*stats,inplace=True)]),
        tt.Compose([tt.RandomCrop(32, padding=4, padding_mode='reflect'),
                    tt.RandomHorizontalFlip(p=0.5), 
                    tt.RandomVerticalFlip(p=0.5),
                 tt.ColorJitter(brightness=0.4, 
                     contrast=0.5,
                     saturation=0.4, 
                     hue=0.3),
                 tt.GaussianBlur(kernel_size=(11, 21), sigma=(5, 50)),
                 tt.ToTensor(), 
                 tt.Normalize(*stats,inplace=True)])]

def get_names():
    return ['no change', 'horizontal flip', '+ vertical flip', 'color jitter and gaussian blur', 'all']

def get_loader(transform):
    basic_tfms = tt.Compose([tt.ToTensor(), tt.Normalize(*stats)])
    train_ds = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
    train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=3, pin_memory=True)

    val_dataset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=basic_tfms)
    valid_dl = DataLoader(val_dataset, batch_size*2, num_workers=3, pin_memory=True)
    train_dl = DeviceDataLoader(train_dl, device)
    valid_dl = DeviceDataLoader(valid_dl, device)
    return train_dl, valid_dl

In [3]:
import torch

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device("cuda")
    # elif torch.backends.mps.is_available():
    #     return torch.device("mps")
    else:
        return torch.device("cpu")
    
def clear_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    elif torch.backends.mps.is_available():
        torch.mps.empty_cache()
    # else:
    #     return torch.device("cpu")
    # appers there's nothing to do here
        
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)
    
device = get_default_device()
print(f"running on {device}")

running on cuda


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ResNet9(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        
        self.conv1 = self.conv_block(in_channels, 64)
        self.conv2 = self.conv_block(64, 128, pool=True)
        self.res1 = nn.Sequential(self.conv_block(128, 128), self.conv_block(128, 128))
        
        self.conv3 = self.conv_block(128, 256, pool=True)
        self.conv4 = self.conv_block(256, 512, pool=True)
        self.res2 = nn.Sequential(self.conv_block(512, 512), self.conv_block(512, 512))
        
        self.classifier = nn.Sequential(nn.MaxPool2d(4), 
                                        nn.Flatten(), 
                                        nn.Linear(512, num_classes))

    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  
        loss = F.cross_entropy(out, labels) 
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    
        loss = F.cross_entropy(out, labels)   
        acc = accuracy(out, labels)           
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))

    def conv_block(self, in_channels, out_channels, pool=False):
        layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
                nn.BatchNorm2d(out_channels), 
                nn.ReLU(inplace=True)]
        if pool: 
            layers.append(nn.MaxPool2d(2))
        return nn.Sequential(*layers)
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

In [5]:
model = to_device(ResNet9(3, 10), device)

In [6]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    # Set up cutom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader),
                                                pct_start=0.3)
    
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            # Gradient clipping
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()
        
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [7]:
epochs = 8
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [8]:
history_map = {}
index = 0
for transformation in get_transformations():
    model = to_device(ResNet9(3, 10), device)
    history = []
    train_dl, valid_dl = get_loader(transformation)
    history += fit_one_cycle(epochs, max_lr, model, train_dl, valid_dl, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)
    history_map[index] = history
    index += 1

Files already downloaded and verified
Files already downloaded and verified
Epoch [0], last_lr: 0.00388, train_loss: 1.4766, val_loss: 1.8734, val_acc: 0.4689
Epoch [1], last_lr: 0.00935, train_loss: 1.0554, val_loss: 1.2884, val_acc: 0.5979
Epoch [2], last_lr: 0.00972, train_loss: 0.8449, val_loss: 0.8291, val_acc: 0.7210
Epoch [3], last_lr: 0.00812, train_loss: 0.5774, val_loss: 0.8934, val_acc: 0.6930
Epoch [4], last_lr: 0.00556, train_loss: 0.3437, val_loss: 0.5566, val_acc: 0.8104
Epoch [5], last_lr: 0.00283, train_loss: 0.2084, val_loss: 0.4759, val_acc: 0.8395
Epoch [6], last_lr: 0.00077, train_loss: 0.0975, val_loss: 0.3825, val_acc: 0.8780
Epoch [7], last_lr: 0.00000, train_loss: 0.0518, val_loss: 0.3555, val_acc: 0.8858
Files already downloaded and verified
Files already downloaded and verified
Epoch [0], last_lr: 0.00388, train_loss: 1.5203, val_loss: 1.7223, val_acc: 0.5007
Epoch [1], last_lr: 0.00935, train_loss: 1.1208, val_loss: 2.3220, val_acc: 0.4592
Epoch [2], last_lr

In [9]:
# comparing different augmentations
import pandas as pd

def compare_final_val_losses():
    # Create a list of final validation losses
    final_val_losses = [history_map[i][-1]['val_loss'] for i in range(len(history_map))]
    final_val_acc = [history_map[i][-1]['val_acc'] for i in range(len(history_map))]

    # Get the transformation names
    names = get_names()

    # Create a pandas DataFrame and display it
    df = pd.DataFrame({
        'Transformation': names,
        'Final Validation Loss': final_val_losses
    })
    print(df)
    print("*"*10)
    df = pd.DataFrame({
        'Transformation': names,
        'Final Validation Accuracy': final_val_acc
    })
    print(df)
    

compare_final_val_losses()

                   Transformation  Final Validation Loss
0                       no change               0.355511
1                 horizontal flip               0.332637
2                 + vertical flip               0.486799
3  color jitter and gaussian blur               4.621833
4                             all               4.211268
**********
                   Transformation  Final Validation Accuracy
0                       no change                   0.885826
1                 horizontal flip                   0.885780
2                 + vertical flip                   0.831345
3  color jitter and gaussian blur                   0.160393
4                             all                   0.182091


In [12]:
# redefine to see if we add 1 to the basic if it does better

def get_loader(transform):
    basic_tfms = tt.Compose([tt.ToTensor(), tt.Normalize(*stats)])
    train_ds_new = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
    train_ds_basic = torchvision.datasets.CIFAR10(root='./data', train=True, transform=basic_tfms, download=False)
    train_dataset = ConcatDataset([train_ds_basic, train_ds_new])
    train_dl = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=3, pin_memory=True)

    val_dataset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=basic_tfms)
    valid_dl = DataLoader(val_dataset, batch_size*2, num_workers=3, pin_memory=True)
    train_dl = DeviceDataLoader(train_dl, device)
    valid_dl = DeviceDataLoader(valid_dl, device)
    return train_dl, valid_dl

In [13]:
history_map = {}
index = 0
for transformation in get_transformations():
    model = to_device(ResNet9(3, 10), device)
    history = []
    train_dl, valid_dl = get_loader(transformation)
    history += fit_one_cycle(epochs, max_lr, model, train_dl, valid_dl, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)
    history_map[index] = history
    index += 1

Files already downloaded and verified
Files already downloaded and verified
Epoch [0], last_lr: 0.00392, train_loss: 1.1231, val_loss: 0.8883, val_acc: 0.6989
Epoch [1], last_lr: 0.00935, train_loss: 0.6968, val_loss: 2.0268, val_acc: 0.5745
Epoch [2], last_lr: 0.00972, train_loss: 0.4486, val_loss: 0.6799, val_acc: 0.7895
Epoch [3], last_lr: 0.00812, train_loss: 0.2017, val_loss: 0.6495, val_acc: 0.8047
Epoch [4], last_lr: 0.00556, train_loss: 0.0635, val_loss: 0.4402, val_acc: 0.8691
Epoch [5], last_lr: 0.00283, train_loss: 0.0096, val_loss: 0.3860, val_acc: 0.8960
Epoch [6], last_lr: 0.00077, train_loss: 0.0011, val_loss: 0.3914, val_acc: 0.8991
Epoch [7], last_lr: 0.00000, train_loss: 0.0008, val_loss: 0.3970, val_acc: 0.8985
Files already downloaded and verified
Files already downloaded and verified
Epoch [0], last_lr: 0.00392, train_loss: 1.1938, val_loss: 1.2161, val_acc: 0.6185
Epoch [1], last_lr: 0.00935, train_loss: 0.7794, val_loss: 1.2021, val_acc: 0.6561
Epoch [2], last_lr

In [14]:
# comparing different augmentations
import pandas as pd

def compare_final_val_losses():
    # Create a list of final validation losses
    final_val_losses = [history_map[i][-1]['val_loss'] for i in range(len(history_map))]
    final_val_acc = [history_map[i][-1]['val_acc'] for i in range(len(history_map))]

    # Get the transformation names
    names = get_names()

    # Create a pandas DataFrame and display it
    df = pd.DataFrame({
        'Basic + Transformation': names,
        'Final Validation Loss': final_val_losses
    })
    print(df)
    print("*"*10)
    df = pd.DataFrame({
        'Basic + Transformation': names,
        'Final Validation Accuracy': final_val_acc
    })
    print(df)
    

compare_final_val_losses()

           Basic + Transformation  Final Validation Loss
0                       no change               0.396951
1                 horizontal flip               0.256883
2                 + vertical flip               0.281133
3  color jitter and gaussian blur               0.364833
4                             all               0.371641
**********
           Basic + Transformation  Final Validation Accuracy
0                       no change                   0.898546
1                 horizontal flip                   0.917098
2                 + vertical flip                   0.906091
3  color jitter and gaussian blur                   0.877467
4                             all                   0.879322


## 